<a href="https://colab.research.google.com/github/SONG-0502/1st-pap/blob/main/svdmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1导入数据

# 导入所需库
import pandas as pd
import numpy as np
import ast  # 用于解析字符串中的 Python 对象（如列表或字典）

# 1.1 加载数据集
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# 查看数据集结构
print("Movies dataset:")
print(movies.head())
print("\nCredits dataset:")
print(credits.head())


# 填充缺失的 'overview' 列为 'No Description'（或空字符串）
movies['overview'] = movies['overview'].fillna('No Description')
# 使用 vote_average 的均值填充 NaN
movies['vote_average'].fillna(movies['vote_average'].mean(), inplace=True)

# 处理缺失的 'vote_average' 列（如果有需要填充的情况）
# 这里假设决定填充缺失的 'vote_average' 列为该列的均值（如果确实存在缺失）
if movies['vote_average'].isnull().sum() > 0:
    mean_vote_average = movies['vote_average'].mean()
    movies['vote_average'] = movies['vote_average'].fillna(mean_vote_average)



Movies dataset:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [

<ipython-input-3-2b7d7ca4dc59>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['vote_average'].fillna(movies['vote_average'].mean(), inplace=True)



Merged dataset:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  


Features column:
0    Action Adventure Fantasy Science Fiction Sam W...
1    Adventure Fantasy Action Johnny Depp Orlando B...
2    Action Adventure Crime Daniel Craig Christoph ...
3    Action Crime Drama Thriller Christian Bale Mic...
4    Action Adventure Science Fiction Taylor Kitsch...
Name: features, dtype: object

Feature matrix shape: (4748, 24795)

Cosine similarity matrix shape: (4748, 4748)

Sample similarity scores for the first movie:
[1.         0.12267912 0.06726728 0.04865043 0.11962972 0.11962972
 0.0239594  0.1903749  0.07161149 0.08492078]


In [12]:
# 1导入数据

# 导入所需库
import pandas as pd
import numpy as np
import ast  # 用于解析字符串中的 Python 对象（如列表或字典）

# 1.1 加载数据集
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# 查看数据集结构
print("Movies dataset:")
print(movies.head())
print("\nCredits dataset:")
print(credits.head())


# 填充缺失的 'overview' 列为 'No Description'（或空字符串）
movies['overview'] = movies['overview'].fillna('No Description')
# 使用 vote_average 的均值填充 NaN
movies['vote_average'].fillna(movies['vote_average'].mean(), inplace=True)

# 处理缺失的 'vote_average' 列（如果有需要填充的情况）
# 这里假设决定填充缺失的 'vote_average' 列为该列的均值（如果确实存在缺失）
if movies['vote_average'].isnull().sum() > 0:
    mean_vote_average = movies['vote_average'].mean()
    movies['vote_average'] = movies['vote_average'].fillna(mean_vote_average)


# 1.2 合并数据集
# credits 中的 movie_id 列与 movies 中的 id 列一致，需重命名后合并
credits.rename(columns={'movie_id': 'id'}, inplace=True)
movies = movies.merge(credits, on='id')

# 查看合并后的数据结构
print("\nMerged dataset:")
print(movies.head())

# 1.3 选择我们需要的列

# 更新要保留的列，将 'title' 替换为 'original_title'
columns_to_keep = ['id', 'original_title', 'genres', 'overview', 'popularity',
                   'vote_average', 'vote_count', 'cast', 'crew']

# 选择相关列
movies = movies[columns_to_keep]

# 重新命名列以保持一致性
movies.rename(columns={'original_title': 'title'}, inplace=True)

# 确认更改
print("\nUpdated dataset with 'title':")
print(movies.head())

# 检查是否还有缺失值
print(movies['vote_average'].isnull().sum())  # 应该输出 0，表示没有缺失值

# 查看数据缺失情况
print("\nMissing values:")
print(movies.isnull().sum())

# 删除缺失值
movies.dropna(inplace=True)

# 1.4 数据格式解析
# 将 genres 和 cast 列从 JSON 格式解析为 Python 列表

# 解析 genres 列
def parse_genres(genres_str):
    genres = ast.literal_eval(genres_str)  # 将字符串转换为列表
    return [genre['name'] for genre in genres]  # 提取每个 genre 的名称

movies['genres'] = movies['genres'].apply(parse_genres)

# 解析 cast 列，提取前3个演员的名字
def parse_cast(cast_str):
    cast = ast.literal_eval(cast_str)  # 将字符串转换为列表
    return [actor['name'] for actor in cast[:3]]  # 提取前3名演员的名字

movies['cast'] = movies['cast'].apply(parse_cast)

# 查看处理后的数据
print("\nProcessed dataset:")
print(movies.head())

# 1.5 进一步清洗
# 删除没有相关信息的电影
movies = movies[movies['genres'].map(len) > 0]
movies = movies[movies['cast'].map(len) > 0]

# 重置索引
movies.reset_index(drop=True, inplace=True)

print("\nFinal cleaned dataset:")
print(movies.head())

# 2特征工程

# 导入所需库
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# 2.1 构建特征列
# 将电影的多个特征（类型、演员、简介）整合为单个字符串
movies['features'] = (
    movies['genres'].apply(lambda x: ' '.join(x)) + ' ' +
    movies['cast'].apply(lambda x: ' '.join(x)) + ' ' +
    movies['overview']
)

# 查看构建的特征
print("\nFeatures column:")
print(movies['features'].head())

# 2.2 文本向量化
# 使用 CountVectorizer 将文本特征转换为稀疏矩阵
vectorizer = CountVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(movies['features'])

# 查看特征矩阵的形状
print("\nFeature matrix shape:", feature_matrix.shape)

# 2.3 计算余弦相似度
# 使用余弦相似度计算电影之间的相似度
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)

# 查看相似度矩阵的形状
print("\nCosine similarity matrix shape:", cosine_sim.shape)

# 查看某一部电影与其他电影的相似度
print("\nSample similarity scores for the first movie:")
print(cosine_sim[0][:10])  # 第一部电影与前10部电影的相似度


# 3. 生成用户评分数据
def generate_user_ratings(movies, num_users, num_ratings):
    """
    根据 vote_average 和 vote_count 创建用户评分数据。
    :param movies: 电影数据集
    :param num_users: 用户数量
    :param num_ratings: 总评分数量
    :return: 用户评分数据 DataFrame
    """
    # 电影的评分平均值和受欢迎程度
    movie_ids = movies['id']
    vote_averages = movies['vote_average']
    vote_counts = movies['vote_count']

    # 计算电影被选择的概率（基于 vote_count，受欢迎的电影更常被选中）
    probabilities = vote_counts / vote_counts.sum()

    # 随机分配用户评分
    ratings = []
    for _ in range(num_ratings):
        user_id = np.random.randint(1, num_users + 1)  # 随机用户
        movie_id = np.random.choice(movie_ids, p=probabilities)  # 按概率选择电影
        movie_idx = movie_ids[movie_ids == movie_id].index[0]

        # 基于电影的 vote_average 生成评分，加一点噪声
        base_rating = vote_averages[movie_idx]
        noise = np.random.normal(0, 0.5)  # 噪声的标准差为 0.5
        rating = max(1, min(5, base_rating / 2 + noise))  # 限制在 [1, 5] 范围内
        ratings.append([user_id, movie_id, round(rating, 1)])  # 保留一位小数

    # 创建 DataFrame
    return pd.DataFrame(ratings, columns=['userId', 'movieId', 'rating'])

# 生成用户评分数据
num_users = 100  # 用户数量
num_ratings = 1000  # 总评分数量
user_ratings = generate_user_ratings(movies, num_users, num_ratings)

# 查看生成的评分数据
print("\nGenerated User Ratings Data:")
print(user_ratings.head())

# 划分训练集和测试集
from sklearn.model_selection import train_test_split
train_ratings, test_ratings = train_test_split(user_ratings, test_size=0.2, random_state=42)

# 查看训练集和测试集的大小
print(f"\nTraining set size: {len(train_ratings)}")
print(f"Testing set size: {len(test_ratings)}")

# 保存到文件（可选）
train_ratings.to_csv('train_ratings.csv', index=False)
test_ratings.to_csv('test_ratings.csv', index=False)

print("\nData processing and splitting completed.")



import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

# 1. 加载数据
train_ratings = pd.read_csv('train_ratings.csv')
test_ratings = pd.read_csv('test_ratings.csv')

# 2. 数据准备
# 创建用户-电影评分矩阵
# Before pivoting, group by 'userId' and 'movieId' and take the average rating.
# 处理训练集，创建用户-电影评分矩阵（缺失值填充为 0）
train_ratings = train_ratings.groupby(['userId', 'movieId'])['rating'].mean().reset_index()
R_df = train_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)


# 获取用户和电影的 ID 列表
user_ids = R_df.index
movie_ids = R_df.columns


# # Model 3: SVD (奇异值分解)
# 1. 获取最受欢迎的前 10 部电影

top_10_svd_movies = movies[['title', 'vote_average']].sort_values(by='vote_average', ascending=False).head(10)

print("Top 10 Movies by SVD:")
print(top_10_svd_movies)

# 2. 获取为每部电影推荐的最相似的前 10 部电影
# Convert R_df to a sparse matrix
R_sparse = csr_matrix(R_df.values)
# Now use R_sparse in svds
U, sigma, Vt = svds(R_sparse, k=50)

sigma = np.diag(sigma)
svd_predictions = np.dot(np.dot(U, sigma), Vt)
svd_predictions_df = pd.DataFrame(svd_predictions, index=user_ids, columns=movie_ids)

def svd_similar_movies(movie_id, top_n=10):
    movie_ratings = svd_predictions_df[movie_id]
    similar_movies = movie_ratings.sort_values(ascending=False).iloc[1:top_n+1]
    return movies[movies['id'].isin(similar_movies.index)][['title', 'vote_average']]

print("Top 10 SVD-based similar movies to:", movies.iloc[0]['title'])
print(svd_similar_movies(movies.iloc[0]['id']))


# 3. 获取为每个用户推荐的最适合的前 10 部电影

def svd_recommend_movies(user_id, top_n=10):
    user_predictions = svd_predictions_df.loc[user_id]
    recommended_movies = user_predictions.sort_values(ascending=False).head(top_n)

    # 确保返回 'id', 'title', 'vote_average', 以及 'genres' 列
    return movies[movies['id'].isin(recommended_movies.index)][['id', 'title', 'vote_average', 'genres']]

print("Top 10 recommended movies for user 1 (SVD):")
print(svd_recommend_movies(user_id=1))



Movies dataset:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [

<ipython-input-12-9383a64ed079>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['vote_average'].fillna(movies['vote_average'].mean(), inplace=True)



Processed dataset:
       id                                     title  \
0   19995                                    Avatar   
1     285  Pirates of the Caribbean: At World's End   
2  206647                                   Spectre   
3   49026                     The Dark Knight Rises   
4   49529                               John Carter   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950

保存数据为 .pkl 文件

In [14]:
#1.1 保存用户-电影评分矩阵（R_df）
import pickle

# 保存用户-电影评分矩阵
def save_rating_matrix(R_df):
    with open('rating_matrix.pkl', 'wb') as f:
        pickle.dump(R_df, f)
    print("评分矩阵已保存为 rating_matrix.pkl")

# 调用保存函数
save_rating_matrix(R_df)

#1.2 保存 SVD 分解结果（U, sigma, Vt）
def save_svd_results(U, sigma, Vt):
    with open('svd_results.pkl', 'wb') as f:
        pickle.dump((U, sigma, Vt), f)
    print("SVD 结果已保存为 svd_results.pkl")

# 调用保存函数
save_svd_results(U, sigma, Vt)

#1.3 保存电影推荐结果（svd_similar_movies 函数的输出）
def save_movie_recommendations(movies, svd_similar_movies):
    movie_recommendations = {}

    # Get movie IDs present in the training data (R_df)
    valid_movie_ids = R_df.columns

    for idx, movie in movies.iterrows():
        movie_id = movie['id']

        # Only get recommendations if the movie ID is in the training data
        if movie_id in valid_movie_ids:
            recommended_movies = svd_similar_movies(movie_id)
            movie_recommendations[movie_id] = recommended_movies

    with open('movie_recommendations.pkl', 'wb') as f:
        pickle.dump(movie_recommendations, f)
    print("电影推荐结果已保存为 movie_recommendations.pkl")

# 调用保存函数
save_movie_recommendations(movies, svd_similar_movies)

#1.4 保存用户推荐结果（svd_recommend_movies 函数的输出）
def save_user_recommendations(svd_recommend_movies, num_users=100):
    user_recommendations = {}

    for user_id in range(1, num_users + 1):
        recommended_movies = svd_recommend_movies(user_id)
        user_recommendations[user_id] = recommended_movies

    with open('user_recommendations.pkl', 'wb') as f:
        pickle.dump(user_recommendations, f)
    print("用户推荐结果已保存为 user_recommendations.pkl")

# 调用保存函数
save_user_recommendations(svd_recommend_movies)


评分矩阵已保存为 rating_matrix.pkl
SVD 结果已保存为 svd_results.pkl
电影推荐结果已保存为 movie_recommendations.pkl
用户推荐结果已保存为 user_recommendations.pkl


In [ ]:
%%writefile app9.py

In [86]:
%%writefile app9.py
import pickle
import streamlit as st
import pandas as pd
import requests
import numpy as np
from time import sleep

# Load saved pkl files
def load_data():
    with open('rating_matrix.pkl', 'rb') as f:
        rating_matrix = pickle.load(f)
    with open('svd_results.pkl', 'rb') as f:
        svd_results = pickle.load(f)  # Assume svd_results is a tuple
    with open('movie_recommendations.pkl', 'rb') as f:
        movie_recommendations = pickle.load(f)
    with open('user_recommendations.pkl', 'rb') as f:
        user_recommendations = pickle.load(f)

    return rating_matrix, svd_results, movie_recommendations, user_recommendations

# Call TMDB API to get movie details
def get_movie_details(movie_id_or_title):
    api_key = 'c1a0fd94adf3e2100677929be4361d07'  # Replace with your TMDB API key
    if isinstance(movie_id_or_title, int):  # If movie_id_or_title is an integer (movie ID)
        url = f"https://api.themoviedb.org/3/movie/{movie_id_or_title}?api_key={api_key}&language=en-US"
    else:  # If it's a movie title
        search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_id_or_title}&language=en-US"
        response = requests.get(search_url)
        search_data = response.json()
        if search_data['results']:
            movie_data = search_data['results'][0]  # Get the first search result
        else:
            return {}

    # Ensure we get the correct details, including poster
    movie_details = {
        'title': movie_data.get('title', 'N/A'),
        'overview': movie_data.get('overview', 'No overview available'),
        'poster_path': movie_data.get('poster_path', ''),
        'release_date': movie_data.get('release_date', 'N/A'),
        'vote_average': movie_data.get('vote_average', 'N/A'),
        'genres': [genre['name'] for genre in movie_data.get('genres', [])] or ['No genres available'],
        'id': movie_data.get('id', None)  # Add the movie TMDB ID
    }

    return movie_details

# Apply styles to the page
def apply_styles():
    st.markdown("""
        <style>
        .css-1d391kg {background-color: #f0f0f5;}
        .css-1y4v6u7 {background-color: #ffffff;}
        .css-1gh9u8e {font-size: 25px; color: #1f77b4;}
        .stButton button {background-color: #1f77b4; color: white; font-size: 18px; border-radius: 5px;}
        .stSelectbox div {background-color: #f1f1f1;}
        .stRadio button {background-color: #2196F3;}
        .stImage {border-radius: 10px; border: 2px solid #1f77b4;}
        </style>
    """, unsafe_allow_html=True)

# Main page layout
def main():
    # Apply page styles
    apply_styles()

    # Set page title
    st.title("Movie Recommendation System")
    st.header("Personalized Recommendations: Based on Your Preferences")

    # Add navigation menu
    menu = ["Home", "Movie Recommendations", "User Recommendations", "Popular Movies", "Rating & Feedback", "About"]
    choice = st.sidebar.radio("Select Page", menu)

    # Load data
    rating_matrix, svd_results, movie_recommendations, user_recommendations = load_data()

    # Display different pages
    if choice == "Home":
        st.subheader("Welcome to the Movie Recommendation System!")
        st.write("""
            This system recommends movies based on your preferences. You can choose different recommendation methods to personalize your experience.
        """)

        # Author Information
        st.write("This system was designed and developed by:")
        st.write("**Song Tianxia**")
        st.write("**Ren Pengcheng**")

    elif choice == "Movie Recommendations":
        recommendation_method = st.radio("Choose Recommendation Method", ("Based on Movies", "Based on Users"))

        if recommendation_method == "Based on Movies":
            # Get all movie titles
            all_movie_titles = []
            for movie_id, recommendations_df in movie_recommendations.items():
                all_movie_titles.extend(recommendations_df['title'].values)
            all_movie_titles = list(set(all_movie_titles))  # Remove duplicates

            # Choose a movie
            movie_title = st.selectbox("Choose a Movie", all_movie_titles)

            if movie_title:
                # Find the movie ID corresponding to the selected title
                selected_movie_id = None
                for movie_id, recommendations_df in movie_recommendations.items():
                    if movie_title in recommendations_df['title'].values:
                        selected_movie_id = movie_id
                        break

                # Get recommended movies for the selected movie
                recommended_movies = movie_recommendations[selected_movie_id]
                st.subheader(f"Recommended Movies for: {movie_title}")
                for index, movie in recommended_movies.iterrows():
                    movie_title = movie['title']
                    details = get_movie_details(movie_title)

                    if details:
                        poster_url = f"https://image.tmdb.org/t/p/w500{details['poster_path']}" if details['poster_path'] else 'https://via.placeholder.com/150'
                        movie_id = details['id']  # Use the TMDB ID for the link
                        if movie_id:
                            # Display movie poster as a clickable image
                            st.image(poster_url, width=150)  # Display the movie poster image directly
                            st.markdown(f"[{details['title']}](https://www.themoviedb.org/movie/{movie_id})")
                        st.write(f"**Title**: {details['title']}")
                        st.write(f"**Rating**: {details['vote_average']} / 10")
                        st.write(f"**Release Date**: {details['release_date']}")
                        st.write(f"**Genres**: {', '.join(details['genres'])}")
                        st.write(f"**Overview**: {details['overview']}")

        elif recommendation_method == "Based on Users":
            user_id = st.selectbox("Choose a User ID", list(user_recommendations.keys()))
            user_recs = user_recommendations.get(user_id, [])
            if isinstance(user_recs, pd.DataFrame):
                st.subheader(f"Movies Recommended for User {user_id}")
                for index, movie in user_recs.iterrows():
                    st.write(f"Recommended Movie: {movie['title']}, Rating: {movie['vote_average']}")
                    details = get_movie_details(movie['title'])
                    if details:
                        poster_url = f"https://image.tmdb.org/t/p/w500{details['poster_path']}" if details['poster_path'] else 'https://via.placeholder.com/150'
                        movie_id = details['id']  # Use the TMDB ID for the link

                        # Display movie poster as a clickable image
                        if movie_id:
                            st.image(poster_url, width=150)  # Display the movie poster image directly
                            st.markdown(f"[{details['title']}](https://www.themoviedb.org/movie/{movie_id})")

                        st.write(f"**Title**: {details['title']}")
                        st.write(f"**Rating**: {details['vote_average']} / 10")
                        st.write(f"**Release Date**: {details['release_date']}")
                        st.write(f"**Genres**: {', '.join(details['genres'])}")
                        st.write(f"**Overview**: {details['overview']}")

    elif choice == "Popular Movies":
        st.subheader("Popular Movies")
        popular_movies_url = "https://api.themoviedb.org/3/movie/popular?api_key=c1a0fd94adf3e2100677929be4361d07&language=en-US&page=1"
        response = requests.get(popular_movies_url)
        popular_data = response.json()

        if 'results' in popular_data:
            for movie in popular_data['results']:
                movie_title = movie['title']
                movie_id = movie['id']
                poster_url = f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie['poster_path'] else 'https://via.placeholder.com/150'
                movie_link = f'<a href="https://www.themoviedb.org/movie/{movie_id}" target="_blank"><img src="{poster_url}" alt="{movie_title}" style="width:150px; border-radius:5px;"></a>'
                st.markdown(f'[![{movie_title}]({poster_url})](https://www.themoviedb.org/movie/{movie_id})')
                st.write(f"**Title**: {movie_title}")
                st.write(f"**Release Date**: {movie['release_date']}")
                st.write(f"**Rating**: {movie['vote_average']} / 10")
                st.write(f"**Overview**: {movie['overview']}")

    elif choice == "Rating & Feedback":
        st.subheader("Rate a Movie")
        movie_title = st.text_input("Enter a Movie Title")
        rating = st.slider("Rate this Movie", 1, 10, 5)
        submit = st.button("Submit Rating")

        if submit:
            st.write(f"Thank you for rating: Movie '{movie_title}' rated {rating} stars!")
            st.write("Your feedback will help improve the recommendation system.")

    elif choice == "About":
        st.subheader("About this System")
        st.write("""
            This system analyzes users' movie preferences and uses recommendation algorithms (such as collaborative filtering and matrix factorization) to recommend movies.
            The system uses TMDB API to retrieve movie data, including posters, ratings, overviews, etc.
        """)
        # Author Information
        st.write("This system was designed and developed by:")
        st.write("**Song Tianxia**")
        st.write("**Ren Pengcheng**")

if __name__ == '__main__':
    main()


Overwriting app9.py


In [87]:
from pyngrok import ngrok

# Stop any existing ngrok tunnels
ngrok.kill()

# Set your authtoken
ngrok.set_auth_token("2qbOSr2wuRpHZoOBEk6GUQ6voiY_5Hkt6cev3Eqks2FqmcbxX") # Replace YOUR_AUTHTOKEN with your actual authtoken

# 使用 ngrok 打开端口
public_url = ngrok.connect(addr=8501)
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://a630-34-145-66-177.ngrok-free.app" -> "http://localhost:8501"
